# 20151230-predict-income-from-census

Related post:  
https://stharrold.github.io/20151230-predict-income-from-census.html

Purpose: Predict total annual household income.

## Initialization

### Imports

In [1]:
cd ~

/home/samuel_harrold


In [2]:
# Import standard packages.
import os
# Import installed packages.
import matplotlib.pyplot as plt
import pandas as pd
# Import local packages.
# TODO: remove autoreload after testing.
# %load_ext autoreload
# %autoreload 2
# IPython magic.
%matplotlib inline

## Globals

In [6]:
path_static = os.path.join(os.path.expanduser(r'~'), r'stharrold.github.io/content/static')
basename = r'20151230-predict-income-from-census'
path_disk = os.path.abspath(r'/mnt/disk-20151227t211000z/')
path_acs = os.path.join(path_disk, r'www2-census-gov/programs-surveys/acs/')
path_hus = os.path.join(path_acs, r'data/pums/2013/5-Year/ss13husa.csv') # 'hus' = 'housing US'
path_ddict = os.path.join(path_acs, r'tech_docs/pums/data_dict/PUMSDataDict13.txt')

## Extract-transform-load

**TODO:**
* Just use pandas. Acknowledge dask.

In [25]:
size_mem = 100e6 # limit ingested data to 100MB for this project
with open(path_hus) as fobj:
    nlines = sum(1 for _ in fobj)
nrows = int(nlines * 100e6 / os.path.getsize(path_hus))

In [27]:
df_acs = pd.read_csv(path_hus, nrows=nrows)
df_acs.head()

,serialno,insp,RT,DIVISION,PUMA00,PUMA10,REGION,ST,ADJHSG,ADJINC,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
0,2009000000015,NaN,H,6,2100,-9,3,1,1086032,1085467,...,26,26,30,25,41,25,26,9,6,21
1,2009000000017,NaN,H,6,1100,-9,3,1,1086032,1085467,...,0,0,0,0,0,0,0,0,0,0
2,2009000000125,0,H,6,2600,-9,3,1,1086032,1085467,...,1,8,8,4,4,1,2,7,4,8
3,2009000000181,400,H,6,1700,-9,3,1,1086032,1085467,...,2,7,3,12,8,3,1,7,8,7
4,2009000000235,NaN,H,6,2100,-9,3,1,1086032,1085467,...,6,8,7,10,4,4,5,1,5,1


In [28]:
percentiles = [0.1587, 0.5000, 0.8413] # +1 std. dev., mean/median, -1 std. dev. for normal dist.
df_acs.describe(percentiles=percentiles, include='all')

,serialno,insp,RT,DIVISION,PUMA00,PUMA10,REGION,ST,ADJHSG,ADJINC,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
count,1.271360e+05,77104.000000,127136,127136.000000,127136.000000,127136.000000,127136.00000,127136.00000,127136.000000,127136.000000,...,127136.000000,127136.000000,127136.000000,127136.000000,127136.000000,127136.000000,127136.000000,127136.000000,127136.000000,127136.000000
unique,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,127136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2.010954e+12,822.495072,NaN,6.203971,771.943879,576.476041,3.06799,1.06799,1041979.809291,1050881.213236,...,18.386035,18.379287,18.363980,18.377934,18.377116,18.382079,18.374363,18.372184,18.377533,18.375818
std,1.406006e+09,763.566754,NaN,0.755190,876.894897,898.711496,0.25173,0.25173,32077.633446,29495.626547,...,20.923660,20.959673,20.751945,20.953753,20.758336,20.786674,20.857573,20.826727,20.669387,21.136350
min,2.009000e+12,0.000000,NaN,6.000000,-9.000000,-9.000000,3.00000,1.00000,1000000.000000,1007549.000000,...,-21.000000,0.000000,0.000000,-5.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15.9%,2.009001e+12,0.000000,NaN,6.000000,-9.000000,-9.000000,3.00000,1.00000,1000000.000000,1007549.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
50%,2.011001e+12,750.000000,NaN,6.000000,400.000000,-9.000000,3.00000,1.00000,1035725.000000,1054614.000000,...,13.000000,12.000000,13.000000,12.000000,13.000000,13.000000,13.000000,13.000000,13.000000,12.000000
84.1%,2.013000e+12,1300.000000,NaN,6.000000,1900.000000,1800.000000,3.00000,1.00000,1086032.000000,1085467.000000,...,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000


In [ ]:
# TODO: resume here. read data dict. verify data validation file.

## Export ipynb to html

In [ ]:
path_ipynb = os.path.join(path_static, basename, basename+'.ipynb')
for template in ['basic', 'full']:
    path_html = os.path.splitext(path_ipynb)[0]+'-'+template+'.html'
    cmd = ['jupyter', 'nbconvert', '--to', 'html', '--template', template, path_ipynb, '--output', path_html]
    print(' '.join(cmd))
    subprocess.run(args=cmd, check=True)